In [23]:
# Validation and Breakdown Before Processing
import pandas as pd

# Load the datasets
stock_df = pd.read_csv('stock.data')
exchange_df = pd.read_csv('exchange.data')
attributes_df = pd.read_csv('attributes.data')
strong_oak_df = pd.read_csv('strong_oak_security_master.csv')

# Print the actual column names of the strong_oak_security_master dataset
print("Columns in strong_oak_security_master.csv:", strong_oak_df.columns)

# Now use the correct column names from the printed output
# Check for missing values
print("Missing values in stock data:")
print(stock_df[['RequestId', 'Symbol', 'QUEUESIP', 'MIC']].isnull().sum())

print("Missing values in strong oak data:")
print(strong_oak_df[['Ticker', 'QUEUESIP', 'Strong Oak Identifier']].isnull().sum())

# Check for missing values in critical columns
print("Missing values in attributes data:")
print(attributes_df[['RequestId', 'Security Name', 'Asset Class', 'Inception Date', 'Return Since Inception']].isnull().sum())



Columns in strong_oak_security_master.csv: Index(['Ticker', 'QUEUESIP', 'Strong Oak Identifier'], dtype='object')
Missing values in stock data:
RequestId       0
Symbol       1580
QUEUESIP      522
MIC             0
dtype: int64
Missing values in strong oak data:
Ticker                   770
QUEUESIP                 368
Strong Oak Identifier      0
dtype: int64
Missing values in attributes data:
RequestId                   0
Security Name               0
Asset Class               514
Inception Date            557
Return Since Inception    550
dtype: int64


In [24]:
# Validation and Breakdown After Processing
import pandas as pd

# Load the processed section1 and section2 data
section1_df = pd.read_csv('Phan_Chenh_section1.csv')
section2_df = pd.read_csv('Phan_Chenh_section2.csv')

# ---- Validation for section1.csv ----
print("Validation for section1.csv:")

# Check for missing values in the key columns (EulerId, MIC, QUEUESIP, Symbol, RequestId)
print("Missing values in section1 data:")
print(section1_df[['EulerId', 'MIC', 'QUEUESIP', 'Symbol', 'RequestId']].isnull().sum())

# Ensure at least one of 'QUEUESIP' or 'Symbol' is populated
invalid_rows = section1_df[section1_df[['QUEUESIP', 'Symbol']].isnull().all(axis=1)]
print(f"Number of rows where both QUEUESIP and Symbol are missing: {len(invalid_rows)}")

# Ensure there are no duplicate EulerId values
duplicate_euler_ids = section1_df['EulerId'].duplicated().sum()
print(f"Number of duplicate EulerIds: {duplicate_euler_ids}")

# Ensure there are no duplicate RequestIds (RequestId should be unique)
duplicate_request_ids = section1_df['RequestId'].duplicated().sum()
print(f"Number of duplicate RequestIds: {duplicate_request_ids}")

# ---- Dynamic MIC Validation ----
# Get the unique MIC values from the section1 dataset
valid_mics = section1_df['MIC'].unique()

# Check for rows with invalid MIC values (rows where MIC is not in the list of valid MICs)
invalid_mics = section1_df[~section1_df['MIC'].isin(valid_mics)]
print(f"Rows with invalid MIC values (outside of valid list): {len(invalid_mics)}")

# ---- Validation for section2.csv ----
print("\nValidation for section2.csv:")

# Check for missing values in AttributeValue (there should be no missing values in AttributeValue)
missing_values_section2 = section2_df[['EulerId', 'AttributeName', 'AttributeValue']].isnull().sum()
print("Missing values in section2 data:")
print(missing_values_section2)

# Ensure all EulerId values in section2 exist in section1 (i.e., the EulerId must match between the two files)
missing_euler_ids_section2 = section2_df[~section2_df['EulerId'].isin(section1_df['EulerId'])]
print(f"Rows in section2 with EulerId not found in section1: {len(missing_euler_ids_section2)}")

# Check for duplicate attribute names for each EulerId (each EulerId should have only one entry for each attribute)
duplicate_attributes = section2_df.groupby(['EulerId', 'AttributeName']).size().reset_index(name='Count')
duplicate_attributes = duplicate_attributes[duplicate_attributes['Count'] > 1]
print(f"Duplicate attribute entries in section2 (EulerId, AttributeName): \n{duplicate_attributes}")

# Ensure that all AttributeNames are valid (must be among the expected attributes)
valid_attributes = ['Asset Class', 'Inception Date', 'Exchange Name', 'Exchange Location', 'Security Name', 'Strong Oak Identifier', 'Return Since Inception']
invalid_attributes_section2 = section2_df[~section2_df['AttributeName'].isin(valid_attributes)]
print(f"Rows with invalid AttributeName in section2: {len(invalid_attributes_section2)}")

# Check if the combination of 'Exchange Name' and 'Exchange Location' is correct (expected country-city format for location)
location_check = section2_df[section2_df['AttributeName'] == 'Exchange Location']
invalid_location_format = location_check[~location_check['AttributeValue'].str.contains(' - ')]
print(f"Rows with invalid Exchange Location format: {len(invalid_location_format)}")


Validation for section1.csv:
Missing values in section1 data:
EulerId         0
MIC             0
QUEUESIP      522
Symbol       1580
RequestId       0
dtype: int64
Number of rows where both QUEUESIP and Symbol are missing: 0
Number of duplicate EulerIds: 0
Number of duplicate RequestIds: 0
Rows with invalid MIC values (outside of valid list): 0

Validation for section2.csv:
Missing values in section2 data:
EulerId           0
AttributeName     0
AttributeValue    0
dtype: int64
Rows in section2 with EulerId not found in section1: 0
Duplicate attribute entries in section2 (EulerId, AttributeName): 
Empty DataFrame
Columns: [EulerId, AttributeName, Count]
Index: []
Rows with invalid AttributeName in section2: 0
Rows with invalid Exchange Location format: 0
